In [1]:
!pip install tiktoken
!pip install plotly
!pip install scipy
!pip install scikit-learn

  Using cached tiktoken-0.6.0-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
Using cached tiktoken-0.6.0-cp311-cp311-win_amd64.whl (798 kB)


In [ ]:
#!pip uninstall openai
!pip install openai==0.28.1

In [1]:
import pandas as pd
#正規表現パッケージ
import re

In [2]:
def remove_newlines(text):
    #正規表現で文字列を置換
    #改行と連続する空白を半角スペースに変換
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r' +', ' ', text)
    return text

In [24]:
#テキストデータを読み込んでデータフレームを返す関数
def text_to_df(data_file):
    #一行分の値を格納する配列を初期化
    texts =[]
    #utf-8でテキストファイルを開く
    with open(data_file, 'r', encoding='utf-8') as file:
        #すべて読み込む
        text = file.read()
        #改行２つを区切りに分割
        sections = text.split('\n\n')
        #項目ごと
        for section in sections:
            #改行で分割
            lines = section.split('\n')
            #配列最初の要素に項目名（例：1. ゲストの歓迎）
            fname = lines[0]
            #配列１番目以降の要素すべてをつなげる
            content = ' '.join(lines[1:])
            #配列を追加
            texts.append([fname, content])
    #配列を元にデータフレーム作成
    df = pd.DataFrame(texts, columns=['fname', 'text'])
    #列内の改行を削除
    df['text'] = df['text'].apply(remove_newlines)
    #データフレームを返す
    return df

In [4]:
#データ読み込み
df = text_to_df('data.txt')
#表をCSVに保存
df.to_csv('scraped.csv', index=False, encoding='utf-8')

In [5]:
df

,fname,text
0,1. ゲストの歓迎,お客様がホテルに到着した際、フレンドリーな笑顔と共に、礼儀正しく、エネルギッシュな挨拶を心掛...
1,2. チェックインとチェックアウト,チェックイン時間は午後3時、チェックアウト時間は午前11時です。早めのチェックインや遅めのチ...
2,3. Wi-Fiと駐車場の案内,全室に無料Wi-Fiが提供されています。接続方法やパスワードについて、確実に説明できるように...
3,4. バリアフリー対応,ユニバーサルルームの配置や設備、特長を理解し、必要に応じてお客様に説明できるようにしましょう...
4,5. ペットの対応,ペット同伴のお客様に対しては、礼儀正しく、しかし明確にペットの同伴はできない旨を伝えましょう...
5,6. ルームサービス,午後11時までのルームサービスを提供しています。ルームサービスメニューの内容を熟知し、お客様...
6,7. 禁煙ポリシーと喫煙室の案内,全客室は禁煙です。しかし、喫煙者のお客様のニーズにも応えるため、喫煙室を1階に設けています。...
7,8. キャンセルポリシー,キャンセル料は、前日までの連絡で宿泊料金の30%、当日のキャンセルで50%、連絡なしの場合は...
8,9. お支払いについて,チェックアウト時にはフロントで現金、クレジットカード、デビットカードによるお支払いをお願いし...
9,10. 常に敬意を持つ,お客様一人一人に敬意を持って接しましょう。お客様に対する礼儀正しさ、思いやり、プロ意識は、ホ...


### 学習データをエンベディング

In [6]:
import pandas as pd
#トークン数管理ライブラリ
import tiktoken
#エンベッディング用ライブラリ
from openai.embeddings_utils import get_embedding

In [25]:
#embedding_model = "text-embedding-3-small"
#モデル、エンコード、最大トークン数
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"
max_tokens = 1500
#先ほどのCSVファイルを読み込みデータフレームへ
df = pd.read_csv('scraped.csv')
df.columns = ['title', 'text']
#n_tokensにトークン数格納
tokenizer = tiktoken.get_encoding(embedding_encoding)
df['n_tokens'] = df.text.apply(lambda x: len(tokenizer.encode(x)))

In [8]:
df

,title,text,n_tokens
0,1. ゲストの歓迎,お客様がホテルに到着した際、フレンドリーな笑顔と共に、礼儀正しく、エネルギッシュな挨拶を心掛...,163
1,2. チェックインとチェックアウト,チェックイン時間は午後3時、チェックアウト時間は午前11時です。早めのチェックインや遅めのチ...,122
2,3. Wi-Fiと駐車場の案内,全室に無料Wi-Fiが提供されています。接続方法やパスワードについて、確実に説明できるように...,113
3,4. バリアフリー対応,ユニバーサルルームの配置や設備、特長を理解し、必要に応じてお客様に説明できるようにしましょう...,130
4,5. ペットの対応,ペット同伴のお客様に対しては、礼儀正しく、しかし明確にペットの同伴はできない旨を伝えましょう...,128
5,6. ルームサービス,午後11時までのルームサービスを提供しています。ルームサービスメニューの内容を熟知し、お客様...,124
6,7. 禁煙ポリシーと喫煙室の案内,全客室は禁煙です。しかし、喫煙者のお客様のニーズにも応えるため、喫煙室を1階に設けています。...,99
7,8. キャンセルポリシー,キャンセル料は、前日までの連絡で宿泊料金の30%、当日のキャンセルで50%、連絡なしの場合は...,103
8,9. お支払いについて,チェックアウト時にはフロントで現金、クレジットカード、デビットカードによるお支払いをお願いし...,133
9,10. 常に敬意を持つ,お客様一人一人に敬意を持って接しましょう。お客様に対する礼儀正しさ、思いやり、プロ意識は、ホ...,114


In [37]:
#
def split_into_many(text, max_tokens = 500):
    #「。」で文を分割
    sentences = text.split('。')
    #分割された文をエンコードしてトークン数を調べる
    n_tokens = [len(tokenizer.encode(" " + sentence)) for sentence in sentences]
    #変数初期化
    chunks = []
    tokens_so_far = 0
    chunk = []
    #文とトークン数をまとめてループ
    for sentence, token in zip(sentences, n_tokens):
        #カウントアップしたトークン数の値が最大トークン数より大きい場合
        if tokens_so_far + token > max_tokens:
            #chunks配列に追加し、chunkとカウンター初期化
            chunks.append(". ".join(chunk) + ".")
            chunk = []
            tokens_so_far = 0
        #１つのトークンが最大トークンより大きい場合は続ける
        if token > max_tokens:
            continue
        #chunkに文を追加
        chunk.append(sentence)
        #トークンを１つカウントアップ
        tokens_so_far += token + 1
    #チャンクに値が入っていたら、chunksに追加
    if chunk:
        chunks.append(". ".join(chunk) + ".")
    #chunks（文章）返す
    return chunks

In [38]:
#データフレームを配列に一時的に変換する用の配列
shortened = []

#データフレームを一行ずつ走査
for row in df.iterrows():
    #値がない場合は続ける
    if row[1]['text'] is None:
        continue

    #トークン数が設定した最大トークン数より大きい場合
    if row[1]['n_tokens'] > max_tokens:
        #文章を分割してshortenedに連結
        shortened += split_into_many(row[1]['text'])
    #それ以外
    else:
        #そのままshortenedに追加
        shortened.append(row[1]['text'])
#shortened配列をデータフレームに列名はtext
df = pd.DataFrame(shortened, columns=['text'])
#トークン数の列を追加
df['n_tokens'] = df.text.apply(lambda x: len(tokenizer.encode(x)))
#エンベッドした列を追加
df['embeddings']=df.text.apply(lambda x: get_embedding(x, engine=embedding_model))
#CSVに保存（永続化データ）
df.to_csv('embeddings.csv')

In [39]:
df

,text,n_tokens,embeddings
0,お客様がホテルに到着した際、フレンドリーな笑顔と共に、礼儀正しく、エネルギッシュな挨拶を心掛...,163,"[-0.002241256181150675, -0.0017336489399895072..."
1,チェックイン時間は午後3時、チェックアウト時間は午前11時です。早めのチェックインや遅めのチ...,122,"[-0.0005805598339065909, 0.004130268469452858,..."
2,全室に無料Wi-Fiが提供されています。接続方法やパスワードについて、確実に説明できるように...,113,"[0.006401557009667158, 0.008329352363944054, 0..."
3,ユニバーサルルームの配置や設備、特長を理解し、必要に応じてお客様に説明できるようにしましょう...,130,"[0.0037083665374666452, 0.0006142923957668245,..."
4,ペット同伴のお客様に対しては、礼儀正しく、しかし明確にペットの同伴はできない旨を伝えましょう...,128,"[0.010119488462805748, -0.00041648291517049074..."
5,午後11時までのルームサービスを提供しています。ルームサービスメニューの内容を熟知し、お客様...,124,"[0.003809709567576647, 0.004822501912713051, -..."
6,全客室は禁煙です。しかし、喫煙者のお客様のニーズにも応えるため、喫煙室を1階に設けています。...,99,"[0.003190160496160388, 0.008686614222824574, -..."
7,キャンセル料は、前日までの連絡で宿泊料金の30%、当日のキャンセルで50%、連絡なしの場合は...,103,"[0.0036050716880708933, 0.011107518337666988, ..."
8,チェックアウト時にはフロントで現金、クレジットカード、デビットカードによるお支払いをお願いし...,133,"[0.001599963172338903, 0.015280676074326038, 0..."
9,お客様一人一人に敬意を持って接しましょう。お客様に対する礼儀正しさ、思いやり、プロ意識は、ホ...,114,"[0.011891301721334457, 0.006994124501943588, 0..."


## LLMの学習した内容だけで応答するチャットボット

In [12]:
import os
import openai

openai.api_key=os.getenv('OPENAI_API_KEY')
#ユーザーに入力指示
print("質問を入力してください。")
#会話履歴用配列
conversation_history=[]

#会話用のループ
while True:
    #ユーザーが入力した値を年数に
    user_input = input()
    #入力値がexitならループを抜けて終了
    if user_input=="exit":
        break

    #会話履歴に辞書型でユーザーの入力値追加
    conversation_history.append(
        {"role":"user", "content":user_input}
    )
    #GPTの応答、会話履歴を渡す
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=conversation_history,
    )
    #GPTのレスポンスを変数に格納
    chatgpt_response=response.choices[0]["message"]["content"]
    #GPTのレスポンスも会話履歴に追加
    conversation_history.append({"role":"assistant", "content":chatgpt_response})
    #GPTのレスポンスを出力表示
    print("ChatGPT:", chatgpt_response)

質問を入力してください。


 茨木県の人口は？


ChatGPT: 茨木県の人口は2021年現在、約80万人です。


 茨木の特産品にはどの様なものがありますか？


ChatGPT: 茨木市には、以下のような特産品があります。

1. 茨木ルーブ - 茨木市をイメージしたカラフルなルーブで、地元の農産物や観光名所をモチーフにした特別デザインが施されています。

2. いばらきやきいも - 茨木市で栽培されたサツマイモを使用した焼き芋。香ばしくて甘い味が特徴です。

3. しいたけ - 茨木市はしいたけの栽培が盛んで、新鮮で美味しいしいたけが特産品として知られています。

4. いばらき杏 - 茨木市で栽培された杏の品種。果肉が瑞々しくて甘い味わいが特徴です。

これらの特産品は、茨木市の地域性や風土を反映したもので、地元の人々や観光客に愛されています。


 他にありますか？


ChatGPT: 申し訳ありませんが、茨木市に関して特に有名な他の特産品はございません。上記の特産品が茨木市の代表的な特産品となります。茨木市周辺で生産される他の地域の特産品もありますが、茨木市独自の特産品は上記のものが主なものです。ご了承ください。


 納豆はどうですか？


ChatGPT: 茨木市では特に納豆が特産品として知られているわけではありませんが、日本全体で広く親しまれている食品です。茨木市でも一般的に食べられており、地元のスーパーや市場でも購入することができます。納豆は日本の代表的な発酵食品であり、栄養価も高く、健康に良いとされています。茨木市でも多くの人に愛されている食品の一つです。


 exit


## 与えた知識を元に回答するチャットボット

In [13]:
import pandas as pd
import openai
import numpy as np
from openai.embeddings_utils import distances_from_embeddings

In [40]:
#質問と学習データを比較して、コンテキストを作成
def create_context(question, df, max_len=1800):
    #質問をベクトル化
    q_embeddings=openai.Embedding.create(input=question, engine='text-embedding-ada-002')['data'][0]['embedding']
    #キーワード間の距離をコサイン近似で求め、データフレームに列を追加
    df['distances']=distances_from_embeddings(q_embeddings, 
                                              df['embeddings'].apply(eval).apply(np.array).values, 
                                              distance_metric='cosine'
                                             )
    #コンテキスト格納用配列
    returns=[]
    #コンテキストの現在の長さ
    cur_len=0
    #データフレームのdistancesでソートしてループ
    for _, row in df.sort_values('distances', ascending=True).iterrows():
        #トークン数＋４でカウントアップ
        cur_len +=row['n_tokens'] + 4
        #cur_lenが指定した最大値より大きければループを抜ける
        if cur_len > max_len:
            break
        #returnsにtextを追加
        returns.append(row['text'])
    #マークダウンの文字列とreturnsを一文に連結して返す
    return "\n\n###\n\n".join(returns)

In [23]:
#質問と会話履歴を引数にGPTに回答させる。
def answer_question(question, conversation_history):
    #学習したRAGデータを読み込む
    df = pd.read_csv('embeddings.csv', encoding="utf-8")
    #質問と学習データを比較してコンテキスト作成する関数（自作関数）
    context = create_context(question, df, max_len=200)
    #プロンプト
    prompt = f"あなたはとあるホテルのスタッフです。コンテキストに基づいて、お客様からの質問に丁寧に答えてください。コンテキストが質問に対して回答できない場合は「わかりません」と答えてください。\n\nコンテキスト: {context}\n\n---\n\n質問: {question}\n回答:"
    #会話履歴にユーザーの入力を追加
    conversation_history.append({"role": "user", "content": prompt})
    #GPTに問い合わせ（例外処理付）
    try:
        #メッセージに会話履歴、temperatureは１で少し自由度持たせる
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=conversation_history,
            temperature=1,
        )
        #GPTの応答を返す
        return response.choices[0]["message"]["content"].strip()
    #例外処理
    except Exception as e:
        #エラーが出たときは空文字を返す
        print(e)
        return ""

In [21]:
import os
import openai

#キーを設定
openai.api_key = os.environ["OPENAI_API_KEY"]

# 最初にメッセージを表示する
print("質問を入力してください")
#会話履歴用配列を初期化
conversation_history = []

#ユーザーがexitと入力するまで無限ループ
while True:
    # ユーザーの入力した文字を変数「user_input」に格納
    user_input = input()

    # ユーザーの入力した文字が「exit」の場合はループを抜ける
    if user_input == "exit":
        break
    #会話履歴にユーザーの質問を追加
    conversation_history.append({"role": "user", "content": user_input})
    #質問に対するGPTの応答を呼び出す（履歴も渡す）
    answer = answer_question(user_input, conversation_history)
    #GPTの回答を出力
    print("ChatGPT:", answer)
    #応答の履歴を会話履歴に追加
    conversation_history.append({"role": "assistant", "content":answer})

質問を入力してください


 駐車場はありますか？


ChatGPT: はい、駐車場を180台分確保しております。ホテル入口付近にございますので、ご到着時にスムーズにご利用いただけます。駐車場は24時間ご利用いただけますので、チェックインからチェックアウトまでご利用いただけます。料金は無料です。


 部屋のタイプはどのようなものがありますか？


ChatGPT: 当ホテルでは、スタンダードルーム、デラックスルーム、スイートルーム、およびユニバーサルルームの4つの部屋タイプをご用意しております。ユニバーサルルームは車椅子を利用するお客様やバリアフリー設備を重視される方に最適です。広めの部屋スペースや専用のアクセシブルバスルームなど、快適な滞在をお約束いたします。他の部屋タイプについても詳細な説明や特長をお知らせいたしますので、お気軽にお問い合わせください。


...

 チェックアウトは何時からですか？


ChatGPT: チェックアウト時間は午前11時となっております。遅めのチェックアウトをご希望される場合は、空室状況によっては対応可能な場合もございますので、ご希望の旨をフロントスタッフにお申し付けください。また、チェックアウト後も一時的にお荷物をお預かりすることも可能ですので、お気軽にお声がけください。何かご質問がございましたら、どうぞお知らせください。


 有難うございました


ChatGPT: どういたしまして。お困りのことがありましたらいつでもお知らせください。お客様のご滞在を快適にお過ごしいただけるよう心がけております。ありがとうございました。


 exit
